# Pilar A1 – Aprendizado Profundo Contínuo

Este notebook é responsável por iniciar a análise exploratória dos dados brutos da base **Ovation**, que serão futuramente curados para alimentar os modelos do Pilar A1 do Framework 3AGD.

---

### Objetivos desta etapa:

1. Verificar estrutura e integridade dos dados
2. Analisar formatos, colunas, datas e registros nulos
3. Definir escopo inicial de modelagem
4. Produzir artefatos auditáveis para curadoria e transformação futura

---


In [3]:
# 🔍 Teste controlado de encoding para leitura segura

from pathlib import Path

falhas = []

for arquivo in arquivos_csv:
    caminho = Path(DATA_PATH) / arquivo
    try:
        # tentativa com 'utf-8'
        pd.read_csv(caminho, sep=",", encoding="utf-8", nrows=5)
        print(f"✅ {arquivo}: lido com utf-8")
    except UnicodeDecodeError:
        try:
            # tentativa com 'latin1'
            pd.read_csv(caminho, sep=",", encoding="latin1", nrows=5)
            print(f"⚠️  {arquivo}: fallback para latin1 (ok)")
        except Exception as e:
            print(f"❌ {arquivo}: falha com ambos os encodings.")
            falhas.append((arquivo, str(e)))

if falhas:
    print("\n🚫 Arquivos com falha crítica:")
    for arq, erro in falhas:
        print(f" - {arq}: {erro}")


✅ Dados WIlson início 2023 1(in).csv: lido com utf-8
✅ Dados Wilson início 2024 1(in).csv: lido com utf-8
✅ Dados Wilson metade final 2023 1(in).csv: lido com utf-8
✅ Dados Wilson metade final 2024 1(in).csv: lido com utf-8


In [6]:
# 🔧 ETAPA 1 – CARREGAMENTO ROBUSTO E CONCILIAÇÃO DOS DADOS OVATION

import os
import pandas as pd

# Caminho fixo dos dados (relativo ao notebook)
DATA_PATH = os.path.join("..", "data", "raw", "ovation")
arquivos_csv = [f for f in os.listdir(DATA_PATH) if f.endswith(".csv")]

print(f"📂 Total de arquivos CSV encontrados: {len(arquivos_csv)}")

dfs = []
falhas = []

# Tenta ler cada arquivo com utf-8, e faz fallback para latin1 se necessário
for arquivo in arquivos_csv:
    caminho = os.path.join(DATA_PATH, arquivo)
    try:
        df = pd.read_csv(caminho, sep=",", encoding="utf-8", low_memory=False)
        print(f"✅ {arquivo}: lido com utf-8")
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(caminho, sep=",", encoding="latin1", low_memory=False)
            print(f"⚠️  {arquivo}: fallback para latin1")
        except Exception as e:
            print(f"❌ {arquivo}: falha ao ler com utf-8 e latin1")
            falhas.append((arquivo, str(e)))
            continue
    df["__source_file"] = arquivo
    dfs.append(df)

# Concatenação dos arquivos válidos
if dfs:
    df_ovation = pd.concat(dfs, ignore_index=True)
    print(f"\n✅ Total de registros concatenados: {len(df_ovation):,}")
    print(f"📊 Total de colunas: {df_ovation.shape[1]}")
    print("📄 Colunas detectadas:")
    print(df_ovation.columns.to_list())
else:
    print("🚫 Nenhum arquivo foi carregado com sucesso.")

# Registro de falhas, se houver
if falhas:
    print("\n🚫 Arquivos que falharam na leitura:")
    for arq, erro in falhas:
        print(f" - {arq}: {erro}")


📂 Total de arquivos CSV encontrados: 4
⚠️  Dados WIlson início 2023 1(in).csv: fallback para latin1
⚠️  Dados Wilson início 2024 1(in).csv: fallback para latin1
⚠️  Dados Wilson metade final 2023 1(in).csv: fallback para latin1
⚠️  Dados Wilson metade final 2024 1(in).csv: fallback para latin1

✅ Total de registros concatenados: 18,072
📊 Total de colunas: 3
📄 Colunas detectadas:
['Monday, January 20, 2025,,,,,,Wilson,,,,,,,Page 1 of 1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,', '__source_file', 'Tuesday, January 21, 2025,,,,,,Wilson,,,,,,Page 1 of 1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,']


In [9]:
import pandas as pd
import os

CAMINHO_ARQUIVOS = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation"

arquivos = [
    "Dados WIlson início 2023 1(in).csv",
    "Dados Wilson metade final 2023 1(in).csv",
    "Dados Wilson início 2024 1(in).csv",
    "Dados Wilson metade final 2024 1(in).csv"
]

dfs = []

for arquivo in arquivos:
    caminho = os.path.join(CAMINHO_ARQUIVOS, arquivo)
    print(f"🔄 Lendo: {arquivo}")
    
    # Agora pula até linha 156, mas considera a 157 como cabeçalho
    df = pd.read_csv(caminho, skiprows=155, header=0, encoding="latin1")
    
    df["__source_file"] = arquivo
    dfs.append(df)

# Concatena
df_concat = pd.concat(dfs, ignore_index=True)

# Salva com o nome definitivo
CAMINHO_SAIDA = os.path.join(CAMINHO_ARQUIVOS, "ovation_dados_concat.csv")
df_concat.to_csv(CAMINHO_SAIDA, index=False, encoding="utf-8")

print(f"\n✅ Arquivo final corrigido salvo em: {CAMINHO_SAIDA}")
print(f"📦 Total de registros: {len(df_concat):,}")
print(f"📊 Total de colunas: {df_concat.shape[1]}")


🔄 Lendo: Dados WIlson início 2023 1(in).csv
🔄 Lendo: Dados Wilson metade final 2023 1(in).csv
🔄 Lendo: Dados Wilson início 2024 1(in).csv
🔄 Lendo: Dados Wilson metade final 2024 1(in).csv

✅ Arquivo final corrigido salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_concat.csv
📦 Total de registros: 17,452
📊 Total de colunas: 3


In [10]:
# Verificar as colunas detectadas após leitura
print("📄 Colunas detectadas no arquivo concatenado:")
print(df_concat.columns.tolist())

# Verificar as 5 primeiras linhas para confirmar estrutura
print("\n🔍 Amostra das primeiras 5 linhas:")
print(df_concat.head())


📄 Colunas detectadas no arquivo concatenado:
[' ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,', '__source_file', ' ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,']

🔍 Amostra das primeiras 5 linhas:
   ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,  \
0  Date Time,,10HFB11CW902.UNIT1@NET1,10HFB12CW90...                                                                                                           
1  06/29/2023 23:00:00,,36.421 ,34.168 ,33.509 ,3...                                                                                                           
2  06/29/2023 22:00:00,,37.313 ,35.276 ,35.290 ,3...                                                       

In [13]:
!pip install tqdm


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [14]:
import os
import re
import pandas as pd
from tqdm import tqdm

# Diretório dos arquivos CSV originais
caminho_arquivos = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation"
arquivos_csv = [
    "Dados WIlson início 2023 1(in).csv",
    "Dados Wilson metade final 2023 1(in).csv",
    "Dados Wilson início 2024 1(in).csv",
    "Dados Wilson metade final 2024 1(in).csv"
]

# Regex para identificar linhas no padrão correto
padrao_linha_valida = re.compile(r'^"\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}",,')

# DataFrame final (sem cabeçalho ainda)
linhas_validas = []

# Loop pelos arquivos
for nome_arquivo in arquivos_csv:
    caminho = os.path.join(caminho_arquivos, nome_arquivo)
    print(f"🔍 Lendo: {nome_arquivo}")
    
    with open(caminho, encoding='latin1') as f:
        for linha in tqdm(f, desc=f"Processando {nome_arquivo}"):
            if padrao_linha_valida.match(linha):
                # Remove quebras de linha e divide por vírgula
                campos = linha.strip().split(',')
                linhas_validas.append(campos)

# Construir DataFrame com nomes de coluna provisórios
max_colunas = max(len(linha) for linha in linhas_validas)
colunas_provisorias = [f"col_{i+1:03}" for i in range(max_colunas)]
df_final = pd.DataFrame(linhas_validas, columns=colunas_provisorias)

# Salvar o arquivo final
caminho_saida = os.path.join(caminho_arquivos, "ovation_dados_filtrados.csv")
df_final.to_csv(caminho_saida, index=False, encoding='utf-8')
print(f"\n✅ Arquivo final salvo com {df_final.shape[0]} linhas e {df_final.shape[1]} colunas.")
print(f"📄 Caminho: {caminho_saida}")


🔍 Lendo: Dados WIlson início 2023 1(in).csv


Processando Dados WIlson início 2023 1(in).csv: 4477it [00:00, 267034.97it/s]


🔍 Lendo: Dados Wilson metade final 2023 1(in).csv


Processando Dados Wilson metade final 2023 1(in).csv: 4549it [00:00, 506931.53it/s]


🔍 Lendo: Dados Wilson início 2024 1(in).csv


Processando Dados Wilson início 2024 1(in).csv: 4501it [00:00, 499922.21it/s]


🔍 Lendo: Dados Wilson metade final 2024 1(in).csv


Processando Dados Wilson metade final 2024 1(in).csv: 4549it [00:00, 454586.13it/s]


ValueError: max() iterable argument is empty

In [16]:
import csv

# Caminho do arquivo de referência
arquivo_referencia = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\Dados WIlson início 2023 1(in).csv"

# Capturar a linha 158 como padrão
linha_referencia = None
with open(arquivo_referencia, "r", encoding="latin1") as f:
    for i, linha in enumerate(f):
        if i == 157:
            campos = list(csv.reader([linha]))[0]
            print(f"🔢 Linha 158 tem {len(campos)} campos")
            print("🧪 Exemplo dos primeiros 5 campos:", campos[:5])
            linha_referencia = campos
            break


🔢 Linha 158 tem 1 campos
🧪 Exemplo dos primeiros 5 campos: ['06/29/2023 23:00:00,,36.421 ,34.168 ,33.509 ,34.521 ,29.797 ,32.795 ,32.676 ,32.603 ,51.230 ,,25.7361 ,,7.9383 ,7.4593 ,5.7430 ,791.737 ,2.140 ,3.124 ,4.168 ,4.544 ,2.982 ,1.961 ,832.467 ,777.996 ,829.101 ,817.945 ,836.842 ,847.498 ,856.979 ,833.293 ,797.198 ,508.047 ,551.398 ,1.115 ,1.245 ,773.662 ,824.498 ,-1.245 ,-42.170 ,9.014 ,830.960 ,839.550 ,864.774 ,-1.430 ,-0.896 ,-1.174 ,306.734 ,314.680 ,-2.828 ,-2.910 ,,,489.291 ,506.136 ,-2.517 ,-2.525 ,-2.152 ,714.335 ,740.825 ,-2.132 ,17.761 ,23.623 ,-39.227 ,1.069 ,799.506 ,1.021 ,183.999 ,774.480 ,151.055 ,-0.072 ,-0.012 ,-0.179 ,0.004 ,77.795 ,74.164 ,131.754 ,132.120 ,15.156 ,15.135 ,10.246 ,9.795 ,1309.138 ,1309.683 ,1302.277 ,1290.948 ,3261.596 ,8.005 ,0.812 ,40.521 ,232.941 ,281.966 ,14.038 ,0.745 ,41.559 ,219.284 ,248.180 ,115.678 ,-5.130 ,-4.217 ,-4.294 ,-4.297 ,-4.371 ,127.739 ,-4.286 ,286.899 ,14.488 ,15.931 ,15.445 ,0.070 ,38.911 ,38.894 ,53.297 ,83.630 ,82.748 ,53

In [17]:
# Caminho do arquivo de referência
arquivo_referencia = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\Dados WIlson início 2023 1(in).csv"

# Capturar e dividir a linha 158
with open(arquivo_referencia, "r", encoding="latin1") as f:
    for i, linha in enumerate(f):
        if i == 157:
            campos = linha.strip().split(',')
            print(f"🔢 Linha 158 tem {len(campos)} campos")
            print("🧪 Primeiros campos:", campos[:5])
            break


🔢 Linha 158 tem 154 campos
🧪 Primeiros campos: ['"06/29/2023 23:00:00', '', '36.421 ', '34.168 ', '33.509 ']


In [18]:
import os
import pandas as pd
from tqdm import tqdm

# 📁 Diretório com os CSVs originais
caminho_arquivos = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation"

# 📝 Arquivos a processar
arquivos_csv = [
    "Dados WIlson início 2023 1(in).csv",
    "Dados Wilson metade final 2023 1(in).csv",
    "Dados Wilson início 2024 1(in).csv",
    "Dados Wilson metade final 2024 1(in).csv"
]

# 📦 Lista para armazenar linhas válidas
linhas_validas = []

# 🔁 Processar linha a linha
for nome_arquivo in arquivos_csv:
    caminho = os.path.join(caminho_arquivos, nome_arquivo)
    print(f"🔍 Lendo: {nome_arquivo}")
    
    with open(caminho, encoding='latin1') as f:
        for linha in tqdm(f, desc=f"Processando {nome_arquivo}"):
            campos = linha.strip().split(',')

            if (
                len(campos) == 154 and                      # exatamente 154 campos
                campos[0].startswith('"') and              # começa com "
                '/' in campos[0] and ':' in campos[0] and  # contém data e hora
                campos[1] == '' and campos[2] != ''        # segue com dois campos: vazio e numérico
            ):
                linhas_validas.append(campos)

# 📐 Cabeçalho provisório
colunas = [f"col_{i+1:03}" for i in range(154)]
df_final = pd.DataFrame(linhas_validas, columns=colunas)

# 💾 Caminho de saída
saida = os.path.join(caminho_arquivos, "ovation_dados_filtrados.csv")
df_final.to_csv(saida, index=False, encoding='utf-8')

# ✅ Feedback
print(f"\n✅ Total de linhas válidas: {len(df_final)}")
print(f"📄 Arquivo final salvo em:\n{saida}")


🔍 Lendo: Dados WIlson início 2023 1(in).csv


Processando Dados WIlson início 2023 1(in).csv: 4477it [00:00, 85032.89it/s]


🔍 Lendo: Dados Wilson metade final 2023 1(in).csv


Processando Dados Wilson metade final 2023 1(in).csv: 4549it [00:00, 105620.32it/s]


🔍 Lendo: Dados Wilson início 2024 1(in).csv


Processando Dados Wilson início 2024 1(in).csv: 4501it [00:00, 121007.12it/s]


🔍 Lendo: Dados Wilson metade final 2024 1(in).csv


Processando Dados Wilson metade final 2024 1(in).csv: 4549it [00:00, 158230.34it/s]



✅ Total de linhas válidas: 8712
📄 Arquivo final salvo em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_filtrados.csv


In [19]:
import os
import pandas as pd
from tqdm import tqdm

# 📁 Diretório com os CSVs
caminho_arquivos = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation"

# 📄 Nomes dos arquivos originais
arquivos_csv = [
    "Dados WIlson início 2023 1(in).csv",
    "Dados Wilson metade final 2023 1(in).csv",
    "Dados Wilson início 2024 1(in).csv",
    "Dados Wilson metade final 2024 1(in).csv"
]

linhas_validas = []

# 🔁 Para cada arquivo...
for nome_arquivo in arquivos_csv:
    caminho = os.path.join(caminho_arquivos, nome_arquivo)
    print(f"\n🔍 Lendo: {nome_arquivo}")
    total_lidas = 0
    validas_arquivo = 0

    with open(caminho, encoding='latin1') as f:
        for linha in tqdm(f, desc=f"Processando {nome_arquivo}"):
            total_lidas += 1
            campos = linha.strip().split(',')

            if (
                len(campos) == 154 and
                campos[0].startswith('"') and
                '/' in campos[0] and ':' in campos[0] and
                campos[1] == '' and campos[2] != ''
            ):
                linhas_validas.append(campos)
                validas_arquivo += 1

    print(f"✅ Linhas válidas em {nome_arquivo}: {validas_arquivo} de {total_lidas}")

# 📐 Cabeçalho numérico
colunas = [f"col_{i+1:03}" for i in range(154)]
df_concat = pd.DataFrame(linhas_validas, columns=colunas)

# 💾 Salvar CSV final
saida = os.path.join(caminho_arquivos, "ovation_dados_filtrados.csv")
df_concat.to_csv(saida, index=False, encoding='utf-8')

print(f"\n📊 Total de linhas válidas dos 4 arquivos: {len(df_concat)}")
print(f"📄 Arquivo final salvo em:\n{saida}")



🔍 Lendo: Dados WIlson início 2023 1(in).csv


Processando Dados WIlson início 2023 1(in).csv: 4477it [00:00, 110859.28it/s]


✅ Linhas válidas em Dados WIlson início 2023 1(in).csv: 4320 de 4477

🔍 Lendo: Dados Wilson metade final 2023 1(in).csv


Processando Dados Wilson metade final 2023 1(in).csv: 4549it [00:00, 91197.52it/s]


✅ Linhas válidas em Dados Wilson metade final 2023 1(in).csv: 4392 de 4549

🔍 Lendo: Dados Wilson início 2024 1(in).csv


Processando Dados Wilson início 2024 1(in).csv: 4501it [00:00, 175077.09it/s]


✅ Linhas válidas em Dados Wilson início 2024 1(in).csv: 0 de 4501

🔍 Lendo: Dados Wilson metade final 2024 1(in).csv


Processando Dados Wilson metade final 2024 1(in).csv: 4549it [00:00, 187762.77it/s]

✅ Linhas válidas em Dados Wilson metade final 2024 1(in).csv: 0 de 4549



📊 Total de linhas válidas dos 4 arquivos: 8712
📄 Arquivo final salvo em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_filtrados.csv


In [20]:
import os
import pandas as pd
from tqdm import tqdm

# 📁 Caminho base
caminho_arquivos = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation"

# 📄 Nome do segundo arquivo
nome_arquivo = "Dados Wilson metade final 2023 1(in).csv"
caminho = os.path.join(caminho_arquivos, nome_arquivo)

# 📦 Armazena as linhas válidas
linhas_validas = []

print(f"\n🔍 Lendo: {nome_arquivo}")
total_lidas = 0
validas = 0

with open(caminho, encoding='latin1') as f:
    for linha in tqdm(f, desc=f"Processando {nome_arquivo}"):
        total_lidas += 1
        campos = linha.strip().split(',')

        if (
            len(campos) == 154 and
            campos[0].startswith('"') and
            '/' in campos[0] and ':' in campos[0] and
            campos[1] == '' and campos[2] != ''
        ):
            linhas_validas.append(campos)
            validas += 1

print(f"📄 Total lidas: {total_lidas}")
print(f"✅ Linhas válidas encontradas: {validas}")

# 🧱 Cabeçalho numérico
colunas = [f"col_{i+1:03}" for i in range(154)]
df_segundo = pd.DataFrame(linhas_validas, columns=colunas)

# 💾 Salvar com nome exclusivo
saida = os.path.join(caminho_arquivos, "ovation_dados_filtrados_2.csv")
df_segundo.to_csv(saida, index=False, encoding='utf-8')

print(f"\n📁 Arquivo salvo em:\n{saida}")



🔍 Lendo: Dados Wilson metade final 2023 1(in).csv


Processando Dados Wilson metade final 2023 1(in).csv: 4549it [00:00, 111432.98it/s]

📄 Total lidas: 4549
✅ Linhas válidas encontradas: 4392



📁 Arquivo salvo em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_filtrados_2.csv


In [21]:
import os
import pandas as pd
from tqdm import tqdm

# 📁 Caminho base
caminho_arquivos = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation"

# 📄 Nome do terceiro arquivo
nome_arquivo = "Dados Wilson início 2024 1(in).csv"
caminho = os.path.join(caminho_arquivos, nome_arquivo)

# 📦 Armazena as linhas válidas
linhas_validas = []

print(f"\n🔍 Lendo: {nome_arquivo}")
total_lidas = 0
validas = 0

with open(caminho, encoding='latin1') as f:
    for linha in tqdm(f, desc=f"Processando {nome_arquivo}"):
        total_lidas += 1
        campos = linha.strip().split(',')

        if (
            len(campos) == 154 and
            campos[0].startswith('"') and
            '/' in campos[0] and ':' in campos[0] and
            campos[1] == '' and campos[2] != ''
        ):
            linhas_validas.append(campos)
            validas += 1

print(f"📄 Total lidas: {total_lidas}")
print(f"✅ Linhas válidas encontradas: {validas}")

# 🧱 Cabeçalho numérico
colunas = [f"col_{i+1:03}" for i in range(154)]
df_terceiro = pd.DataFrame(linhas_validas, columns=colunas)

# 💾 Salvar com nome exclusivo
saida = os.path.join(caminho_arquivos, "ovation_dados_filtrados_3.csv")
df_terceiro.to_csv(saida, index=False, encoding='utf-8')

print(f"\n📁 Arquivo salvo em:\n{saida}")



🔍 Lendo: Dados Wilson início 2024 1(in).csv


Processando Dados Wilson início 2024 1(in).csv: 4501it [00:00, 134638.18it/s]

📄 Total lidas: 4501
✅ Linhas válidas encontradas: 0

📁 Arquivo salvo em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_filtrados_3.csv


In [22]:
import os
import csv

# 📄 Caminho do arquivo 3
arquivo_3 = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\Dados Wilson início 2024 1(in).csv"

# 🔍 Ler linha 158
linha_alvo = None
with open(arquivo_3, "r", encoding="latin1") as f:
    for i, linha in enumerate(f):
        if i == 157:  # linha 158 (começando do 0)
            linha_alvo = linha
            break

# 🧪 Analisar a linha
if linha_alvo:
    campos = list(csv.reader([linha_alvo]))[0]
    print(f"🔢 Linha 158 tem {len(campos)} campos")
    print(f"🧪 Primeiros 5 campos: {campos[:5]}")
else:
    print("❌ Linha 158 não encontrada.")


🔢 Linha 158 tem 1 campos
🧪 Primeiros 5 campos: ['06/29/2024 23:00:00,,43.159 ,43.232 ,42.504 ,40.275 ,38.938 ,40.948 ,42.074 ,40.705 ,56.580 ,24.4613 ,,6.9846 ,6.7926 ,5.4346 ,825.693 ,1.872 ,2.723 ,3.550 ,4.111 ,2.591 ,1.699 ,888.215 ,830.387 ,876.699 ,855.016 ,871.531 ,847.171 ,887.133 ,879.912 ,832.673 ,508.322 ,509.192 ,0.822 ,1.024 ,817.877 ,870.971 ,95.526 ,38.672 ,8.512 ,871.988 ,887.034 ,915.060 ,-1.774 ,-1.410 ,-1.505 ,306.882 ,314.102 ,-3.426 ,-3.483 ,,,510.356 ,538.025 ,-2.986 ,-2.988 ,-2.508 ,747.937 ,787.535 ,-2.574 ,18.564 ,25.556 ,-43.302 ,1.225 ,817.236 ,1.175 ,189.907 ,899.749 ,155.677 ,0.680 ,-0.009 ,-0.394 ,0.001 ,149.817 ,133.704 ,300.668 ,286.478 ,19.578 ,19.543 ,9.456 ,9.887 ,11.071 ,11.207 ,11.902 ,8.217 ,3020.736 ,4.366 ,1.140 ,47.548 ,286.194 ,323.867 ,17.166 ,1.054 ,50.706 ,268.112 ,302.258 ,115.031 ,-6.163 ,-4.821 ,-5.065 ,-5.149 ,-5.304 ,129.059 ,-5.176 ,273.900 ,18.592 ,16.086 ,15.216 ,0.070 ,40.284 ,41.692 ,55.676 ,78.905 ,78.144 ,55.388 ,5.772 ,0.409 ,77.

In [23]:
# 📄 Caminho do arquivo 3
arquivo_3 = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\Dados Wilson início 2024 1(in).csv"

# 🔍 Ler linha 158 com .split(',')
linha_alvo = None
with open(arquivo_3, "r", encoding="latin1") as f:
    for i, linha in enumerate(f):
        if i == 157:
            linha_alvo = linha
            break

# ✅ Reanalisar com split(',')
if linha_alvo:
    campos = linha_alvo.strip().split(',')
    print(f"🔢 Linha 158 tem {len(campos)} campos (com split)")
    print(f"🧪 Primeiros 5 campos: {campos[:5]}")
else:
    print("❌ Linha 158 não encontrada.")


🔢 Linha 158 tem 153 campos (com split)
🧪 Primeiros 5 campos: ['"06/29/2024 23:00:00', '', '43.159 ', '43.232 ', '42.504 ']


In [24]:
import os
import pandas as pd
from tqdm import tqdm

# 📁 Caminho do arquivo 3
caminho_arquivos = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation"
nome_arquivo = "Dados Wilson início 2024 1(in).csv"
caminho = os.path.join(caminho_arquivos, nome_arquivo)

# 📦 Lista de linhas válidas
linhas_validas = []

print(f"\n🔍 Lendo: {nome_arquivo}")
total_lidas = 0
validas = 0

with open(caminho, encoding='latin1') as f:
    for linha in tqdm(f, desc=f"Processando {nome_arquivo}"):
        total_lidas += 1
        campos = linha.strip().split(',')

        if (
            len(campos) in [153, 154] and
            campos[0].startswith('"') and
            '/' in campos[0] and ':' in campos[0] and
            campos[1] == '' and campos[2] != ''
        ):
            linhas_validas.append(campos)
            validas += 1

print(f"\n📄 Total lidas: {total_lidas}")
print(f"✅ Linhas válidas encontradas: {validas}")

# 🧱 Cabeçalho numérico para 154 colunas
colunas = [f"col_{i+1:03}" for i in range(154)]

# 📐 Ajustar linhas com 153 campos para ter 154
for i in range(len(linhas_validas)):
    if len(linhas_validas[i]) == 153:
        linhas_validas[i].append('')  # adicionar coluna em branco no final

# 🧱 Construir DataFrame
df_ajustado = pd.DataFrame(linhas_validas, columns=colunas)

# 💾 Caminho de saída
saida = os.path.join(caminho_arquivos, "ovation_dados_filtrados_3.csv")
df_ajustado.to_csv(saida, index=False, encoding='utf-8')

print(f"\n📁 Arquivo salvo em:\n{saida}")



🔍 Lendo: Dados Wilson início 2024 1(in).csv


Processando Dados Wilson início 2024 1(in).csv: 4501it [00:00, 84381.76it/s]


📄 Total lidas: 4501
✅ Linhas válidas encontradas: 4344

📁 Arquivo salvo em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_filtrados_3.csv


In [25]:
import os
import pandas as pd
from tqdm import tqdm

# 📁 Caminho do arquivo 4
caminho_arquivos = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation"
nome_arquivo = "Dados Wilson metade final 2024 1(in).csv"
caminho = os.path.join(caminho_arquivos, nome_arquivo)

# 📦 Lista de linhas válidas
linhas_validas = []

print(f"\n🔍 Lendo: {nome_arquivo}")
total_lidas = 0
validas = 0

with open(caminho, encoding='latin1') as f:
    for linha in tqdm(f, desc=f"Processando {nome_arquivo}"):
        total_lidas += 1
        campos = linha.strip().split(',')

        if (
            len(campos) in [153, 154] and
            campos[0].startswith('"') and
            '/' in campos[0] and ':' in campos[0] and
            campos[1] == '' and campos[2] != ''
        ):
            linhas_validas.append(campos)
            validas += 1

print(f"\n📄 Total lidas: {total_lidas}")
print(f"✅ Linhas válidas encontradas: {validas}")

# 🧱 Cabeçalho numérico para 154 colunas
colunas = [f"col_{i+1:03}" for i in range(154)]

# 📐 Ajustar linhas com 153 campos para ter 154
for i in range(len(linhas_validas)):
    if len(linhas_validas[i]) == 153:
        linhas_validas[i].append('')  # adicionar coluna em branco no final

# 🧱 Construir DataFrame
df_ajustado = pd.DataFrame(linhas_validas, columns=colunas)

# 💾 Caminho de saída
saida = os.path.join(caminho_arquivos, "ovation_dados_filtrados_4.csv")
df_ajustado.to_csv(saida, index=False, encoding='utf-8')

print(f"\n📁 Arquivo salvo em:\n{saida}")



🔍 Lendo: Dados Wilson metade final 2024 1(in).csv


Processando Dados Wilson metade final 2024 1(in).csv: 4549it [00:00, 99970.60it/s]


📄 Total lidas: 4549
✅ Linhas válidas encontradas: 4392

📁 Arquivo salvo em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_filtrados_4.csv


In [26]:
import os
import pandas as pd
import numpy as np

# Caminho base dos arquivos já filtrados
base_path = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation"
arquivos = [
    "ovation_dados_filtrados_1.csv",
    "ovation_dados_filtrados_2.csv",
    "ovation_dados_filtrados_3.csv",
    "ovation_dados_filtrados_4.csv"
]

diagnostico = []

for nome_arquivo in arquivos:
    caminho = os.path.join(base_path, nome_arquivo)
    df = pd.read_csv(caminho, encoding="utf-8")

    # Amostragem de 10 linhas distribuídas ao longo do dataset
    amostras = df.iloc[np.linspace(0, len(df) - 1, 10, dtype=int)].copy()

    # Contar quantos campos não estão vazios por linha (exceto o timestamp)
    nao_vazios_por_linha = (amostras.iloc[:, 1:] != '').sum(axis=1)

    diagnostico.append({
        "arquivo": nome_arquivo,
        "linhas_amostradas": len(amostras),
        "total_colunas": df.shape[1],
        "media_num_colunas_nao_vazias": nao_vazios_por_linha.mean(),
        "desvio_padrao": nao_vazios_por_linha.std(),
        "min_colunas_nao_vazias": nao_vazios_por_linha.min(),
        "max_colunas_nao_vazias": nao_vazios_por_linha.max()
    })

# Resultado final
df_resultado = pd.DataFrame(diagnostico)
print("\n📊 Diagnóstico Comparativo entre os Arquivos:")
display(df_resultado)


C:\Users\wilso\AppData\Local\Temp\ipykernel_25252\2786267645.py:18: DtypeWarning: Columns (2,3,5,6,7,8,9,10,12,14,15,16,33,42,43,44,46,47,63,64,69,75,76,77,78,83,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99,108,109,110,111,112,122,129,130,132,133,138,141,149,150) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho, encoding="utf-8")
C:\Users\wilso\AppData\Local\Temp\ipykernel_25252\2786267645.py:18: DtypeWarning: Columns (2,3,6,7,8,9,10,12,14,15,16,33,42,43,44,46,47,63,64,69,75,76,77,78,83,84,85,86,89,91,92,93,94,95,96,97,98,99,108,109,110,111,112,122,129,130,132,133,140,141,149,150) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho, encoding="utf-8")
C:\Users\wilso\AppData\Local\Temp\ipykernel_25252\2786267645.py:18: DtypeWarning: Columns (5,10,11,13,14,15,44,46,62,68,76,82,83,84,85,96,97,107,108,121,129,131,132,138,140,141,148,149) have mixed types. Specify dtype option on import or se


📊 Diagnóstico Comparativo entre os Arquivos:


C:\Users\wilso\AppData\Local\Temp\ipykernel_25252\2786267645.py:18: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,134,135,136,137,138,139,140,142,143,144,145,146,147,148,149,150,151) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho, encoding="utf-8")


,arquivo,linhas_amostradas,total_colunas,media_num_colunas_nao_vazias,desvio_padrao,min_colunas_nao_vazias,max_colunas_nao_vazias
0,ovation_dados_filtrados_1.csv,10,154,153.0,0.0,153,153
1,ovation_dados_filtrados_2.csv,10,154,153.0,0.0,153,153
2,ovation_dados_filtrados_3.csv,10,154,153.0,0.0,153,153
3,ovation_dados_filtrados_4.csv,10,154,153.0,0.0,153,153


In [27]:
import os
import pandas as pd

# Caminho base onde estão os arquivos filtrados
base_path = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation"

# Arquivos validados
arquivos_filtrados = [
    "ovation_dados_filtrados_1.csv",
    "ovation_dados_filtrados_2.csv",
    "ovation_dados_filtrados_3.csv",
    "ovation_dados_filtrados_4.csv"
]

# Lista para armazenar os DataFrames
dfs = []

# Carregar todos os arquivos e adicionar à lista
for nome_arquivo in arquivos_filtrados:
    caminho = os.path.join(base_path, nome_arquivo)
    print(f"📥 Lendo: {nome_arquivo}")
    df = pd.read_csv(caminho, encoding="utf-8")
    dfs.append(df)

# Concatenar todos os DataFrames
df_final = pd.concat(dfs, ignore_index=True)

# Caminho de salvamento
arquivo_saida = os.path.join(base_path, "ovation_dados_curado.csv")

# Salvar resultado final
df_final.to_csv(arquivo_saida, index=False, encoding="utf-8")
print(f"\n✅ Arquivo final salvo em:\n{arquivo_saida}")
print(f"📦 Total de registros concatenados: {len(df_final)}")
print(f"📊 Total de colunas: {df_final.shape[1]}")


📥 Lendo: ovation_dados_filtrados_1.csv
📥 Lendo: ovation_dados_filtrados_2.csv


C:\Users\wilso\AppData\Local\Temp\ipykernel_25252\190084973.py:22: DtypeWarning: Columns (2,3,5,6,7,8,9,10,12,14,15,16,33,42,43,44,46,47,63,64,69,75,76,77,78,83,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99,108,109,110,111,112,122,129,130,132,133,138,141,149,150) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho, encoding="utf-8")
C:\Users\wilso\AppData\Local\Temp\ipykernel_25252\190084973.py:22: DtypeWarning: Columns (2,3,6,7,8,9,10,12,14,15,16,33,42,43,44,46,47,63,64,69,75,76,77,78,83,84,85,86,89,91,92,93,94,95,96,97,98,99,108,109,110,111,112,122,129,130,132,133,140,141,149,150) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho, encoding="utf-8")
C:\Users\wilso\AppData\Local\Temp\ipykernel_25252\190084973.py:22: DtypeWarning: Columns (5,10,11,13,14,15,44,46,62,68,76,82,83,84,85,96,97,107,108,121,129,131,132,138,140,141,148,149) have mixed types. Specify dtype option on import or set l

📥 Lendo: ovation_dados_filtrados_3.csv
📥 Lendo: ovation_dados_filtrados_4.csv


C:\Users\wilso\AppData\Local\Temp\ipykernel_25252\190084973.py:22: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,134,135,136,137,138,139,140,142,143,144,145,146,147,148,149,150,151) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho, encoding="utf-8")



✅ Arquivo final salvo em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado.csv
📦 Total de registros concatenados: 21840
📊 Total de colunas: 154


In [33]:
import pandas as pd

# Caminho do arquivo real com os campos
CAMINHO_CAMPOS = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_campos.csv"

# Diagnóstico inicial sem header e sem skiprows
df_diagnostico = pd.read_csv(CAMINHO_CAMPOS, encoding="latin1", header=None)

# Mostrar dimensões e as primeiras 20 linhas
print(f"📄 Formato detectado: {df_diagnostico.shape[0]} linhas × {df_diagnostico.shape[1]} colunas\n")
display(df_diagnostico.head(20))


📄 Formato detectado: 625 linhas × 3 colunas



,0,1,2
0,linha,conteudo,arquivo_origem
1,1,"""Monday, January 20, 2025,,,,,,Wilson,,,,,,,Pa...",Dados WIlson inÃ­cio 2023 1(in).csv
2,2,"""Start Time : 01/01/2023 00:00:00,,,,,,,,,,,,,...",Dados WIlson inÃ­cio 2023 1(in).csv
3,3,"""End Time : 06/30/2023 00:00:00,,,,,,,,,,,,,,,...",Dados WIlson inÃ­cio 2023 1(in).csv
4,4,""" ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...",Dados WIlson inÃ­cio 2023 1(in).csv
5,5,""",G,Point Name,,Historian,Processing Type,Desc...",Dados WIlson inÃ­cio 2023 1(in).csv
6,6,"""1,On,(A) 10HFB11CW902.UNIT1@NET1,,Auto Histor...",Dados WIlson inÃ­cio 2023 1(in).csv
7,7,"""2,On,(A) 10HFB12CW902.UNIT1@NET1,,Auto Histor...",Dados WIlson inÃ­cio 2023 1(in).csv
8,8,"""3,On,(A) 10HFB21CW902.UNIT1@NET1,,Auto Histor...",Dados WIlson inÃ­cio 2023 1(in).csv
9,9,"""4,On,(A) 10HFB22CW902.UNIT1@NET1,,Auto Histor...",Dados WIlson inÃ­cio 2023 1(in).csv


In [34]:
import pandas as pd
import re
from unidecode import unidecode

# Caminho original
CAMINHO_CAMPOS = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_campos.csv"

# Lê o arquivo bruto
df_raw = pd.read_csv(CAMINHO_CAMPOS, encoding='latin1')

# Extrai a coluna com os dados e remove aspas
linhas_csv = df_raw.iloc[5:, 1].dropna().str.replace('"', '')

# Separa os campos CSV internamente
dados_splitados = linhas_csv.apply(lambda x: x.split(','))

# Constrói um DataFrame
df_campos = pd.DataFrame(dados_splitados.tolist())

# Renomeia campos úteis com base na estrutura esperada
df_campos.columns = [
    "idx", "status", "point_name", "_1", "historian", "proc_type", "descricao",
    "_2", "_3", "end_value", "unidade", "low_scale", "high_scale"
] + [f"extra_{i}" for i in range(1, df_campos.shape[1] - 13 + 1)]

# Remove colunas não utilizadas
df_limpo = df_campos[["point_name", "descricao", "unidade"]].copy()

# Gera nomes curtos com base na descrição
def gerar_nome_curto(texto):
    texto = unidecode(str(texto))                 # remove acentos
    texto = texto.lower()                         # lowercase
    texto = re.sub(r"[^a-z0-9]+", "_", texto)     # troca por underscores
    texto = re.sub(r"_+", "_", texto).strip("_")  # limpa múltiplos _
    return texto

df_limpo["nome_curto"] = df_limpo["descricao"].apply(gerar_nome_curto)

# Salva o mapeamento final
CAMINHO_SAIDA = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_mapeamento_colunas.csv"
df_limpo.to_csv(CAMINHO_SAIDA, index=False, encoding="utf-8")

# Diagnóstico final
print("✅ Mapeamento de nomes curtos gerado com sucesso!")
print(f"📁 Arquivo salvo em: {CAMINHO_SAIDA}")
print(f"🔢 Total de variáveis mapeadas: {len(df_limpo)}")
display(df_limpo.head(10))


✅ Mapeamento de nomes curtos gerado com sucesso!
📁 Arquivo salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_mapeamento_colunas.csv
🔢 Total de variáveis mapeadas: 619


,point_name,descricao,unidade,nome_curto
0,(A) 10HFB11CW902.UNIT1@NET1,FLW OUTPUT OF BELT COAL FDR A1,t/h,flw_output_of_belt_coal_fdr_a1
1,(A) 10HFB12CW902.UNIT1@NET1,FLW OUTPUT OF BELT COAL FDR A2,t/h,flw_output_of_belt_coal_fdr_a2
2,(A) 10HFB21CW902.UNIT1@NET1,FLW OUTPUT OF BELT COAL FDR B1,t/h,flw_output_of_belt_coal_fdr_b1
3,(A) 10HFB22CW902.UNIT1@NET1,FLW OUTPUT OF BELT COAL FDR B2,t/h,flw_output_of_belt_coal_fdr_b2
4,(A) 10HFB23CW902.UNIT1@NET1,FLW OUTPUT OF BELT COAL FDR B3,t/h,flw_output_of_belt_coal_fdr_b3
5,(A) 10HFB24CW902.UNIT1@NET1,FLW OUTPUT OF BELT COAL FDR B4,t/h,flw_output_of_belt_coal_fdr_b4
6,(A) 10HFB31CW902.UNIT1@NET1,FLW OUTPUT OF BELT COAL FDR C1,t/h,flw_output_of_belt_coal_fdr_c1
7,(A) 10HFB32CW902.UNIT1@NET1,FLW OUTPUT OF BELT COAL FDR C2,t/h,flw_output_of_belt_coal_fdr_c2
8,(A) 10PORC-AR-PRIMARIO.UNIT1@NET1,PORCENTAGEM DE AR PRIMARIO,%,porcentagem_de_ar_primario
9,(A) UPPER-AIR-PERC-FLOW.UNIT1@NET1,UPPER-PERC DO AR TOTAL,%,upper_perc_do_ar_total


In [35]:
import pandas as pd

# Caminhos dos arquivos
CAMINHO_DADOS = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado.csv"
CAMINHO_MAPEAMENTO = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_mapeamento_colunas.csv"
CAMINHO_SAIDA = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_renomeado.csv"

# Carrega os dados e o mapeamento
df_dados = pd.read_csv(CAMINHO_DADOS, encoding="utf-8", header=None)
df_mapeamento = pd.read_csv(CAMINHO_MAPEAMENTO)

# Substitui os nomes de coluna (apenas as colunas com variáveis conhecidas)
colunas_renomeadas = ["datahora"] + df_mapeamento["nome_curto"].tolist()
df_dados.columns = colunas_renomeadas[:df_dados.shape[1]]

# Salva o novo CSV com cabeçalhos
df_dados.to_csv(CAMINHO_SAIDA, index=False, encoding="utf-8")

# Diagnóstico
print("✅ Arquivo com cabeçalhos renomeados salvo com sucesso!")
print(f"📁 Local: {CAMINHO_SAIDA}")
print(f"📦 Registros: {len(df_dados)}")
print(f"🔠 Colunas: {df_dados.shape[1]}")
display(df_dados.head(2))


C:\Users\wilso\AppData\Local\Temp\ipykernel_25252\3072358010.py:9: DtypeWarning: Columns (1,2,3,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,134,135,136,137,138,140,142,143,144,145,146,147,148,149,150,151,152,153) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dados = pd.read_csv(CAMINHO_DADOS, encoding="utf-8", header=None)


✅ Arquivo com cabeçalhos renomeados salvo com sucesso!
📁 Local: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_renomeado.csv
📦 Registros: 21841
🔠 Colunas: 154


,datahora,flw_output_of_belt_coal_fdr_a1,flw_output_of_belt_coal_fdr_a2,flw_output_of_belt_coal_fdr_b1,flw_output_of_belt_coal_fdr_b2,flw_output_of_belt_coal_fdr_b3,flw_output_of_belt_coal_fdr_b4,flw_output_of_belt_coal_fdr_c1,flw_output_of_belt_coal_fdr_c2,porcentagem_de_ar_primario,...,temp_of_3_lph_outl_wtr,prs_of_condensate_pump_a_outl,prs_of_condensate_pump_a_inlet,prs_of_condensate_pump_b_inlet,prs_of_condensate_pump_b_outl,temp_of_cnd_pump_outl_header,prs_of_cnd_pump_outl_header,NaN,NaN,NaN
0,col_001,col_002,col_003,col_004,col_005,col_006,col_007,col_008,col_009,col_010,...,col_145,col_146,col_147,col_148,col_149,col_150,col_151,col_152,col_153,col_154
1,"""06/29/2023 23:00:00",NaN,36.421,34.168,33.509,34.521,29.797,32.795,32.676,32.603,...,254.181,224.011,151.055,114.222,3.439,-72.340,-70.775,-0.037,33.596,"3.411 """


In [40]:
import pandas as pd
import re
from unidecode import unidecode

# Caminho do arquivo com estrutura correta
CAMINHO_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_campos_formatado_simples.csv"

# 1. Carregar base
df = pd.read_csv(CAMINHO_BASE, encoding="utf-8")

# 2. Função para gerar nome curto padronizado com unidade
def gerar_nome_curto(desc, unidade):
    base = unidecode(str(desc)).lower()
    base = re.sub(r"[^a-z0-9]+", "_", base).strip("_")
    unidade = unidecode(str(unidade)).lower()
    unidade = re.sub(r"[^a-z0-9]+", "_", unidade).strip("_")
    return f"{base}_{unidade}"

# 3. Aplicar a transformação
df["nome_curto_com_unidade"] = df.apply(
    lambda row: gerar_nome_curto(row["descricao"], row["unidade"]), axis=1
)

# 4. Selecionar colunas relevantes e salvar
df_saida = df[["point_name", "descricao", "unidade", "low_scale", "nome_curto_com_unidade"]]
CAMINHO_SAIDA = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_mapeado_nome_unidade.csv"
df_saida.to_csv(CAMINHO_SAIDA, index=False)

print(f"✅ Arquivo salvo com sucesso em:\n{CAMINHO_SAIDA}")


✅ Arquivo salvo com sucesso em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_mapeado_nome_unidade.csv


In [44]:
import pandas as pd

# === Caminhos de entrada e saída
CAMINHO_CURADO = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado.csv"
CAMINHO_MAPEADO = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_mapeado_nome_unidade.csv"
CAMINHO_SAIDA = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_renomeado.csv"

# === 1. Carregar datasets
df = pd.read_csv(CAMINHO_CURADO, encoding="utf-8", low_memory=False)
df_map = pd.read_csv(CAMINHO_MAPEADO, encoding="utf-8")

# === 2. Separar colunas reais (de sensores) da de data
col_data = df.columns[0]
col_sensores = df.columns[1:]

# === 3. Verificar e completar o mapeamento, se necessário
nomes_disponiveis = df_map["nome_curto_com_unidade"].tolist()
faltam = len(col_sensores) - len(nomes_disponiveis)

if faltam > 0:
    nomes_ficticios = [f"variavel_ficticia_{str(i+1).zfill(3)}" for i in range(faltam)]
    nomes_finais = nomes_disponiveis + nomes_ficticios
else:
    nomes_finais = nomes_disponiveis[:len(col_sensores)]

# === 4. Aplicar renomeação
mapeamento = {
    original: novo for original, novo in zip(col_sensores, nomes_finais)
}
df = df.rename(columns=mapeamento)
df = df.rename(columns={col_data: "data_hora"})

# === 5. Exportar resultado
df.to_csv(CAMINHO_SAIDA, index=False)
print("✅ Renomeação aplicada com nomes reais e fictícios.")
print(f"📁 Arquivo salvo em: {CAMINHO_SAIDA}")
print(f"📊 Total de colunas renomeadas: {len(mapeamento)} + 1 (data_hora)")


✅ Renomeação aplicada com nomes reais e fictícios.
📁 Arquivo salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_renomeado.csv
📊 Total de colunas renomeadas: 153 + 1 (data_hora)


In [1]:
import pandas as pd

CAMINHO_CURADO = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado.csv"

# Carregar apenas o cabeçalho
with open(CAMINHO_CURADO, "r", encoding="utf-8") as f:
    primeira_linha = f.readline()

# Separar e imprimir as colunas
colunas = primeira_linha.strip().split(",")
print(f"🔢 Total de colunas: {len(colunas)}\n")
for i, nome in enumerate(colunas):
    print(f"{i:03}: {nome}")


🔢 Total de colunas: 154

000: col_001
001: col_002
002: col_003
003: col_004
004: col_005
005: col_006
006: col_007
007: col_008
008: col_009
009: col_010
010: col_011
011: col_012
012: col_013
013: col_014
014: col_015
015: col_016
016: col_017
017: col_018
018: col_019
019: col_020
020: col_021
021: col_022
022: col_023
023: col_024
024: col_025
025: col_026
026: col_027
027: col_028
028: col_029
029: col_030
030: col_031
031: col_032
032: col_033
033: col_034
034: col_035
035: col_036
036: col_037
037: col_038
038: col_039
039: col_040
040: col_041
041: col_042
042: col_043
043: col_044
044: col_045
045: col_046
046: col_047
047: col_048
048: col_049
049: col_050
050: col_051
051: col_052
052: col_053
053: col_054
054: col_055
055: col_056
056: col_057
057: col_058
058: col_059
059: col_060
060: col_061
061: col_062
062: col_063
063: col_064
064: col_065
065: col_066
066: col_067
067: col_068
068: col_069
069: col_070
070: col_071
071: col_072
072: col_073
073: col_074
074: col_075
